### Final Team Project : Multi-Agent Financial Analysis System (LangChain : Multi-Agent)

In [69]:
!pip install langgraph-supervisor langchain-openai
!pip install -U langchain-community
! pip install langchain_groq
! pip install groq
! pip install tools


#### 1. Library Import and setting up Environment :-

In [70]:
import os
import time
import torch
import json
import pandas as pd
import numpy as np
import yfinance as yf
import matplotlib.pyplot as plt

from langchain_groq import ChatGroq
from langchain.tools import Tool, StructuredTool
from langchain.document_loaders.web_base import WebBaseLoader
from langchain_community.tools.yahoo_finance_news import YahooFinanceNewsTool
# Agent builder
from langgraph.prebuilt import create_react_agent



In [71]:
GROQ_API_Key = "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"
# Set up GROQ API Key as Environmental variable
os.environ["GROQ_API_KEY"] = GROQ_API_Key

#### 2. Defining Specialized Agent Functions :-   

Market News Analysis :

In [72]:
def Format_results(docs, query):
  title_content_list = []
  # Iterate through the loaded web pages and format their respective titles and content
  for doc in docs:
    title = doc.metadata.get('title','No title available')
    page_content = doc.page_content.strip() if query in doc.page_content else "" # Removing unnecessary trailing/leading spaces
    title_content = f"{title}:{page_content}\n"
    title_content_list.append(title_content)
  # Join formatted content and titles into a single string
  return "\n".join(title_content_list)

In [73]:
# Retrieving Market news
# Function to retrieve Stock symbol and stock related news from Yahoo Finance
def Financial_News(ticker, n_search_results=2):
  """ Simulating fetching financial news for a given stock """
  # Retrieve the news from Yahoo Finance
  links = []
  try:
    # Create a yfinance.Ticker object for the specified ticker
    company = yf.Ticker(ticker)
    # Retrieving news articles of type "STORY" and storing their respective links
    links = [n["link"] for n in company.news if n["type"] == "STORY"]
    print(f"Links are retrieved and stored successfully")
  except:
    print(f"No news found from Yahoo Finance")

  # Create a WebBaseLoader to load web pages using the collected links
  loader = WebBaseLoader(links)
  docs = loader.load()
  # Formatting the results by combining titles and page content
  data = Format_results(docs, ticker)
  print(f"Completed retrieving news for {ticker}")
  return data


Financial and Market Data Analysis :

In [74]:
def Financial_Statements(ticker):
  """ Here we simulate fetching stock market data for a given stock symbol. """
  # Creating a Ticker object for the specified stock ticker
  company = yf.Ticker(ticker)

  # Modules to fetch company's balance sheet, cash flow and income statement data
  balance_sheet_statement = company.balance_sheet
  cash_flow_statement = company.cash_flow
  income_statement = company.income_stmt

  # Set up the file name with csv pre-fix
  csv_file_prefix = f"{ticker}_financial_"

  # Retrieve the Stock price data
  stock_data = yf.download(ticker, period='1y', interval='1d')

  # Saving stock price data to the CSV file
  data_csv_filename = csv_file_prefix + "stock_data.csv"
  stock_data.to_csv(data_csv_filename)

  # Save financial statements to the CSV file
  balance_sheet_statement_csv_filename = csv_file_prefix + "balance_sheet_statement.csv"
  cash_flow_statement_csv_filename = csv_file_prefix + "cash_flow_statement.csv"
  income_statement_csv_filename = csv_file_prefix + "income_statement.csv"

  # Save the data from balance sheet, cash flow, and income statement
  balance_sheet_statement.to_csv(balance_sheet_statement_csv_filename)
  cash_flow_statement.to_csv(cash_flow_statement_csv_filename)
  income_statement.to_csv(income_statement_csv_filename)

  print(f"Financial Statements and Stock Price data are saved in respective CSV files")
  return data_csv_filename, balance_sheet_statement_csv_filename, cash_flow_statement_csv_filename, income_statement_csv_filename


Quantitative Analysis :

In [75]:
# quant_updated.py
# ------------------------------------------------------------
# Full updated code: enhanced fundamentals + quant add-ons
# ------------------------------------------------------------
from __future__ import annotations

import os
from dataclasses import dataclass
from typing import Dict, Any, Optional, List, Tuple

import numpy as np
import pandas as pd
import yfinance as yf
import statsmodels.api as sm
import matplotlib.pyplot as plt

# ============================================================
# 1) FUNDAMENTALS — upgraded, non-breaking
# ============================================================

def Fundamental_Analysis_Statements(ticker: str, prefer_period: str = "annual", trend_periods: int = 8) -> Dict[str, Any]:
    """
    Compute fundamentals & trends (safe yfinance access).
    Adds: ROA, ROIC (approx), EBITDA/FCF margins, Quick Ratio proxy, Interest Coverage,
          Debt/EBITDA, EV/EBITDA, YoY growth metrics and robust charts.
    Keeps your original keys: info, financials, balance_sheet, cash_flow, ratios, notes.
    Adds: trends
    """
    company = yf.Ticker(ticker)
    result = {
        'ticker': ticker,
        'info': {},
        'financials': None,
        'balance_sheet': None,
        'cash_flow': None,
        'ratios': {},
        'trends': {},
        'notes': []
    }

    # -- info
    try:
        info = company.info or {}
        result['info'] = info
    except Exception as e:
        info = {}
        result['notes'].append(f"Unable to retrieve ticker.info: {e}")

    # -- choose statements (prefer quarterly/annual with fallback)
    def pick_statements(prefer: str):
        inc = bs = cf = None
        try:
            if prefer.lower().startswith("q"):
                inc = company.quarterly_income_stmt
                bs = company.quarterly_balance_sheet
                cf = company.quarterly_cashflow
            else:
                inc = company.income_stmt
                bs = company.balance_sheet
                cf = company.cash_flow
        except Exception:
            pass
        if inc is None or inc.empty:
            try: inc = company.income_stmt
            except Exception: pass
        if bs is None or bs.empty:
            try: bs = company.balance_sheet
            except Exception: pass
        if cf is None or cf.empty:
            try: cf = company.cash_flow
            except Exception: pass
        return inc, bs, cf

    income_statement, balance_sheet, cash_flow = pick_statements(prefer_period)
    result['financials'] = income_statement
    result['balance_sheet'] = balance_sheet
    result['cash_flow'] = cash_flow

    if (income_statement is None or income_statement.empty) or (balance_sheet is None or balance_sheet.empty):
        result['notes'].append("Income Statement or Balance Sheet is empty; ratios limited.")
    if cash_flow is None or cash_flow.empty:
        result['notes'].append("Cash Flow statement empty; FCF metrics limited.")

    # -- helpers
    def s_float(x):
        try: return float(x)
        except Exception: return np.nan

    def latest(series: Optional[pd.Series]) -> float:
        if series is None or len(series) == 0: return np.nan
        return s_float(series.iloc[0])

    def find_row(df: Optional[pd.DataFrame], candidates: List[str]) -> Optional[pd.Series]:
        if df is None or df.empty: return None
        for label in candidates:
            if label in df.index:
                return df.loc[label]
        index_lower = {str(idx).lower(): idx for idx in df.index}
        for label in candidates:
            key = label.lower()
            for lidx, orig in index_lower.items():
                if key in lidx:
                    return df.loc[orig]
        return None

    # common aliases
    REV_ALIASES   = ['Total Revenue', 'Revenues', 'Revenue', 'TotalRevenue']
    NI_ALIASES    = ['Net Income', 'NetIncome', 'Net Income Common Stockholders', 'Net Income Applicable To Common Shares']
    OPEX_ALIASES  = ['Total Operating Expenses', 'Operating Expense', 'Operating Expenses']
    OPINC_ALIASES = ['Operating Income', 'OperatingIncome']
    EBITDA_AL     = ['EBITDA', 'Ebitda']
    EBIT_AL       = ['EBIT', 'Ebit', 'Operating Income']

    EQ_ALIASES  = ['Total Stockholder Equity', 'Total Equity Gross Minority Interest', 'Stockholders Equity']
    TL_ALIASES  = ['Total Liabilities', 'Total Liabilities Net Minority Interest', 'Liabilities']
    CA_ALIASES  = ['Current Assets', 'Total Current Assets']
    CL_ALIASES  = ['Current Liabilities', 'Total Current Liabilities']
    CASH_ALIASES= ['Cash And Cash Equivalents', 'Cash And Cash Equivalents, At Carrying Value', 'Cash']
    DEBT_ALIASES= ['Total Debt', 'Long Term Debt And Capital Lease Obligation', 'LongTermDebt', 'Total Debt Net Minority Interest']
    INTEXP_AL   = ['Interest Expense', 'InterestExpense']
    TA_ALIASES  = ['Total Assets', 'TotalAssets']

    # extract rows
    rev_series   = find_row(income_statement, REV_ALIASES)
    ni_series    = find_row(income_statement, NI_ALIASES)
    opex_series  = find_row(income_statement, OPEX_ALIASES)
    opinc_series = find_row(income_statement, OPINC_ALIASES)
    ebitda_series= find_row(income_statement, EBITDA_AL)
    ebit_series  = find_row(income_statement, EBIT_AL)

    equity_series= find_row(balance_sheet, EQ_ALIASES)
    tl_series    = find_row(balance_sheet, TL_ALIASES)
    ca_series    = find_row(balance_sheet, CA_ALIASES)
    cl_series    = find_row(balance_sheet, CL_ALIASES)
    cash_series  = find_row(balance_sheet, CASH_ALIASES)
    debt_series  = find_row(balance_sheet, DEBT_ALIASES)
    ta_series    = find_row(balance_sheet, TA_ALIASES)

    intexp_series = (find_row(income_statement, INTEXP_AL) or find_row(cash_flow, INTEXP_AL))

    revenue            = latest(rev_series)
    net_income         = latest(ni_series)
    operating_expenses = latest(opex_series)
    operating_income   = latest(opinc_series)
    ebitda             = latest(ebitda_series)
    ebit               = latest(ebit_series)

    shareholders_equity= latest(equity_series)
    total_liabilities  = latest(tl_series)
    current_assets     = latest(ca_series)
    current_liabilities= latest(cl_series)
    cash               = latest(cash_series)
    total_debt         = latest(debt_series)
    total_assets       = latest(ta_series)
    interest_expense   = abs(latest(intexp_series))  # expense usually negative

    # FCF ~ OCF - CapEx (approx)
    fcf_series = None
    try:
        ocf = find_row(cash_flow, ['Total Cash From Operating Activities', 'Operating Cash Flow', 'Net Cash Provided By Operating Activities'])
        capex = find_row(cash_flow, ['Capital Expenditures', 'Investments In Property, Plant, And Equipment'])
        if ocf is not None and capex is not None:
            fcf_series = ocf - capex
    except Exception:
        pass
    fcf = latest(fcf_series)

    # ---- ratios (yours + added)
    r: Dict[str, Any] = {}
    r['marketCap']    = result['info'].get('marketCap')
    r['trailingPE']   = result['info'].get('trailingPE')
    r['currentPrice'] = result['info'].get('currentPrice')
    r['trailingEps']  = result['info'].get('trailingEps')
    r['pe_ratio']     = (r['currentPrice'] / r['trailingEps']) if r.get('currentPrice') and r.get('trailingEps') else None

    # ROE
    r['ROE'] = float(net_income / shareholders_equity) * 100 if pd.notna(net_income) and pd.notna(shareholders_equity) and shareholders_equity != 0 else None

    # Op margin (prefer operating income)
    if pd.notna(revenue) and revenue != 0:
        if pd.notna(operating_income):
            r['operating_margin'] = float(operating_income / revenue) * 100
        elif pd.notna(operating_expenses):
            r['operating_margin'] = float((revenue - operating_expenses) / revenue) * 100
        else:
            r['operating_margin'] = None
        r['net_profit_margin'] = float(net_income / revenue) * 100 if pd.notna(net_income) else None
    else:
        r['operating_margin'] = None
        r['net_profit_margin'] = None

    # Liquidity & leverage
    r['current_ratio']     = float(current_assets / current_liabilities) if pd.notna(current_assets) and pd.notna(current_liabilities) and current_liabilities != 0 else None
    r['quick_ratio_proxy'] = float(cash / current_liabilities) if pd.notna(cash) and pd.notna(current_liabilities) and current_liabilities != 0 else None
    r['debt_to_equity_ratio'] = float(total_liabilities / shareholders_equity) if pd.notna(total_liabilities) and pd.notna(shareholders_equity) and shareholders_equity != 0 else None

    # Added: ROA
    r['ROA'] = float(net_income / total_assets) * 100 if pd.notna(net_income) and pd.notna(total_assets) and total_assets != 0 else None

    # Added: EBITDA & FCF margins
    r['ebitda_margin'] = float(ebitda / revenue) * 100 if pd.notna(ebitda) and pd.notna(revenue) and revenue != 0 else None
    r['fcf_margin']    = float(fcf / revenue) * 100 if pd.notna(fcf) and pd.notna(revenue) and revenue != 0 else None

    # Added: Interest coverage
    r['interest_coverage'] = float(ebit / interest_expense) if pd.notna(ebit) and pd.notna(interest_expense) and interest_expense != 0 else None

    # Added: Debt/EBITDA
    r['debt_to_ebitda'] = float(total_debt / ebitda) if pd.notna(total_debt) and pd.notna(ebitda) and ebitda not in (0, np.nan) else None

    # Added: EV/EBITDA (rough)
    ev = None
    if r.get('marketCap') is not None:
        ev = float(r['marketCap'] + (total_debt if pd.notna(total_debt) else 0) - (cash if pd.notna(cash) else 0))
    r['ev_to_ebitda'] = float(ev / ebitda) if (ev is not None and pd.notna(ebitda) and ebitda not in (0, np.nan)) else None

    # Added: ROIC (approx)
    eff_tax = result['info'].get('taxRate', 0.21 if result['info'] else 0.21)
    nopat = (ebit if pd.notna(ebit) else np.nan)
    nopat = nopat * (1 - eff_tax) if pd.notna(nopat) else np.nan
    invested_capital = (total_debt if pd.notna(total_debt) else 0) + (shareholders_equity if pd.notna(shareholders_equity) else 0) - (cash if pd.notna(cash) else 0)
    r['ROIC'] = float(nopat / invested_capital) * 100 if pd.notna(nopat) and invested_capital not in (0, np.nan) else None

    result['ratios'] = r

    # ---- trends / growth (last N periods)
    def sort_cols(df: pd.DataFrame) -> pd.DataFrame:
        cols = list(df.columns)
        try:
            dts = pd.to_datetime(cols)
            return df[dts.sort_values().astype(str)]
        except Exception:
            return df

    try:
        inc_sorted = sort_cols(income_statement) if income_statement is not None else None
        bs_sorted  = sort_cols(balance_sheet) if balance_sheet is not None else None

        if inc_sorted is not None and not inc_sorted.empty:
            rev_hist = find_row(inc_sorted, REV_ALIASES)
            ni_hist  = find_row(inc_sorted, NI_ALIASES)

            opm_hist = None
            if rev_hist is not None:
                if opinc_series is not None:
                    op_hist = find_row(inc_sorted, OPINC_ALIASES)
                    if op_hist is not None:
                        opm_hist = (op_hist / rev_hist) * 100
                elif opex_series is not None:
                    opex_hist = find_row(inc_sorted, OPEX_ALIASES)
                    if opex_hist is not None:
                        opm_hist = ((rev_hist - opex_hist) / rev_hist) * 100

            if rev_hist is not None:
                result['trends']['revenue'] = rev_hist.iloc[:trend_periods].to_dict()
                if len(rev_hist) >= 5:
                    base = rev_hist.iloc[4]
                    result['trends']['revenue_yoy_growth'] = float((rev_hist.iloc[0] - base) / abs(base)) if base not in (0, np.nan) else None
            if ni_hist is not None:
                result['trends']['net_income'] = ni_hist.iloc[:trend_periods].to_dict()
                if len(ni_hist) >= 5:
                    base = ni_hist.iloc[4]
                    result['trends']['net_income_yoy_growth'] = float((ni_hist.iloc[0] - base) / abs(base)) if base not in (0, np.nan) else None
            if opm_hist is not None:
                result['trends']['operating_margin_series'] = opm_hist.iloc[:trend_periods].to_dict()

        if bs_sorted is not None and not bs_sorted.empty:
            eq_hist = find_row(bs_sorted, EQ_ALIASES)
            if eq_hist is not None:
                result['trends']['equity'] = eq_hist.iloc[:trend_periods].to_dict()

    except Exception as e:
        result['notes'].append(f"Trend computation warning: {e}")

    # ---- charts (skip gracefully if missing)
    try:
        if ni_series is not None and len(ni_series) > 0:
            ni_series.iloc[:trend_periods].plot(kind='bar', figsize=(6, 5))
            plt.title(f'{ticker} Net Income (last {min(trend_periods, len(ni_series))} periods)')
            plt.xlabel('Period'); plt.ylabel('Net Income ($)')
            plt.tight_layout(); plt.savefig(f'{ticker}_net_income_trend.png'); plt.close()

        if 'operating_margin_series' in result['trends']:
            pd.Series(result['trends']['operating_margin_series']).plot(kind='bar', figsize=(6, 5))
            plt.title(f'{ticker} Operating Margin (%)')
            plt.xlabel('Period'); plt.ylabel('Operating Margin (%)')
            plt.tight_layout(); plt.savefig(f'{ticker}_operating_margin_trend.png'); plt.close()

        if ni_series is not None and equity_series is not None:
            common = ni_series.index.intersection(equity_series.index)
            if len(common) > 0:
                roe_series = (ni_series[common] / equity_series[common]) * 100
                roe_series.iloc[:trend_periods].plot(kind='bar', figsize=(6, 5))
                plt.title(f'{ticker} ROE (%)'); plt.xlabel('Period'); plt.ylabel('ROE (%)')
                plt.tight_layout(); plt.savefig(f'{ticker}_roe_trend.png'); plt.close()

        if ni_series is not None and rev_series is not None:
            common = ni_series.index.intersection(rev_series.index)
            if len(common) > 0:
                npm_series = (ni_series[common] / rev_series[common]) * 100
                npm_series.iloc[:trend_periods].plot(kind='bar', figsize=(6, 5))
                plt.title(f'{ticker} Net Profit Margin (%)')
                plt.xlabel('Period'); plt.ylabel('Net Profit Margin (%)')
                plt.tight_layout(); plt.savefig(f'{ticker}_net_profit_margin_trend.png'); plt.close()
    except Exception as e:
        result['notes'].append(f"Plotting warning: {e}")

    return result


# ============================================================
# 2) QUANT ADD-ONS — price-based analytics toolkit
# ============================================================

# Returns
def simple_returns(close: pd.Series) -> pd.Series:
    s = pd.to_numeric(close, errors="coerce").dropna()
    s.index = pd.to_datetime(s.index)
    return s.pct_change()

def log_returns(close: pd.Series) -> pd.Series:
    s = pd.to_numeric(close, errors="coerce").dropna()
    s.index = pd.to_datetime(s.index)
    return np.log(s).diff()

# Rolling vol & Sharpe
def rolling_volatility(returns: pd.Series, window: int = 20, annualize: bool = True) -> pd.Series:
    vol = returns.rolling(window).std(ddof=1)
    return vol * np.sqrt(252) if annualize else vol

def rolling_sharpe(returns: pd.Series, window: int = 60, rf_annual: float = 0.0) -> pd.Series:
    rf_daily = (1 + rf_annual)**(1/252) - 1
    excess = returns - rf_daily
    m = excess.rolling(window).mean()
    s = excess.rolling(window).std(ddof=1)
    return np.sqrt(252) * (m / (s + 1e-12))

# Risk metrics
def downside_deviation(returns: pd.Series) -> float:
    d = returns[returns < 0]
    return float(np.sqrt((d**2).mean())) if not d.empty else np.nan

def sharpe_sortino(returns: pd.Series, rf_annual: float = 0.0) -> Dict[str, float]:
    r = returns.dropna()
    if r.empty: return {"sharpe": np.nan, "sortino": np.nan}
    rf_daily = (1 + rf_annual)**(1/252) - 1
    excess = r - rf_daily
    sharpe = np.sqrt(252) * (excess.mean() / (excess.std(ddof=1) + 1e-12))
    dd = r[r < 0]
    sortino = np.sqrt(252) * (excess.mean() / (dd.std(ddof=1) + 1e-12)) if not dd.empty else np.nan
    return {"sharpe": float(sharpe), "sortino": float(sortino)}

def annualized_return_vol(returns: pd.Series) -> Dict[str, float]:
    r = returns.dropna()
    if r.empty: return {"ann_return": np.nan, "ann_vol": np.nan}
    ann_ret = (1 + r.mean())**252 - 1
    ann_vol = r.std(ddof=1) * np.sqrt(252)
    return {"ann_return": float(ann_ret), "ann_vol": float(ann_vol)}

def drawdown_series(returns: pd.Series) -> pd.Series:
    eq = (1 + returns.fillna(0)).cumprod()
    peak = eq.cummax()
    return (eq / peak) - 1.0

def max_drawdown_info(returns: pd.Series) -> Dict[str, Any]:
    dd = drawdown_series(returns)
    if dd.empty:
        return {"max_drawdown": np.nan, "start": None, "trough": None, "recovery": None}
    trough = dd.idxmin()
    min_dd = float(dd.loc[trough])
    eq = (1 + returns.fillna(0)).cumprod()
    peak_curve = eq.cummax()
    start_candidates = eq[eq == peak_curve].index
    start_candidates = start_candidates[start_candidates <= trough]
    start = start_candidates[-1] if len(start_candidates) else None
    post = dd[dd.index > trough]
    recov = post[post >= 0].index.min() if not post.empty else None
    return {"max_drawdown": min_dd, "start": start, "trough": trough, "recovery": recov}

def value_at_risk(returns: pd.Series, q: float = 0.05) -> float:
    r = returns.dropna()
    return float(np.percentile(r, q*100)) if not r.empty else np.nan

def conditional_var(returns: pd.Series, q: float = 0.05) -> float:
    r = returns.dropna()
    if r.empty: return np.nan
    var = np.percentile(r, q*100)
    tail = r[r <= var]
    return float(tail.mean()) if not tail.empty else np.nan

def omega_ratio(returns: pd.Series, threshold: float = 0.0) -> float:
    r = returns.dropna()
    above = (r - threshold).clip(lower=0).sum()
    below = (threshold - r).clip(lower=0).sum()
    return float(above / (below + 1e-12)) if not r.empty else np.nan

def tail_ratio(returns: pd.Series) -> float:
    r = returns.dropna()
    pos = r[r > 0].abs().mean() if (r > 0).any() else np.nan
    neg = r[r < 0].abs().mean() if (r < 0).any() else np.nan
    return float(pos / neg) if (isinstance(pos, float) and isinstance(neg, float) and neg) else np.nan

def calmar_ratio(returns: pd.Series) -> float:
    ann = annualized_return_vol(returns)["ann_return"]
    mdd = drawdown_series(returns).min()
    return float(ann / abs(mdd)) if (pd.notna(ann) and pd.notna(mdd) and mdd < 0) else np.nan

# Technicals
def bollinger_bands(close: pd.Series, window: int = 20, num_std: float = 2.0) -> pd.DataFrame:
    m = close.rolling(window).mean()
    s = close.rolling(window).std(ddof=1)
    return pd.DataFrame({"middle": m, "upper": m + num_std*s, "lower": m - num_std*s})

def average_true_range(ohlc: pd.DataFrame, window: int = 14) -> pd.Series:
    h, l, c = ohlc["High"], ohlc["Low"], ohlc["Close"]
    tr = pd.concat([(h - l).abs(), (h - c.shift()).abs(), (l - c.shift()).abs()], axis=1).max(axis=1)
    return tr.rolling(window).mean()

def momentum_12m_1m(close: pd.Series) -> float:
    s = close.dropna()
    if len(s) < 252: return np.nan
    r_12m = s.iloc[-1] / s.iloc[-252] - 1
    r_1m  = s.iloc[-1] / s.iloc[-21]  - 1
    return float((1 + r_12m) / (1 + r_1m) - 1)

def zscore_last(series: pd.Series, window: int = 60) -> float:
    s = series.dropna()
    if len(s) < window: return np.nan
    m = s.rolling(window).mean().iloc[-1]
    v = s.rolling(window).std(ddof=1).iloc[-1]
    return float((s.iloc[-1] - m) / (v + 1e-12))

# CAPM beta/alpha
def capm_beta_alpha(asset_returns: pd.Series, bench_returns: pd.Series) -> Dict[str, float]:
    df = pd.concat([asset_returns, bench_returns], axis=1).dropna()
    if df.shape[0] < 30:
        return {"beta": np.nan, "alpha_annual": np.nan}
    y = df.iloc[:, 0]
    x = sm.add_constant(df.iloc[:, 1])
    model = sm.OLS(y, x).fit()
    beta = float(model.params.iloc[1])
    alpha_daily = float(model.params.iloc[0])
    alpha_annual = (1 + alpha_daily)**252 - 1
    return {"beta": beta, "alpha_annual": alpha_annual}

def rolling_beta(asset_returns: pd.Series, bench_returns: pd.Series, window: int = 60) -> pd.Series:
    df = pd.concat([asset_returns, bench_returns], axis=1).dropna()
    if df.shape[0] < window:
        return pd.Series(dtype=float)
    betas, idx = [], []
    for i in range(window, df.shape[0] + 1):
        sub = df.iloc[i-window:i]
        y = sub.iloc[:, 0]; x = sm.add_constant(sub.iloc[:, 1])
        res = sm.OLS(y, x).fit()
        betas.append(float(res.params.iloc[1])); idx.append(sub.index[-1])
    return pd.Series(betas, index=idx, name="beta")

# Correlation / Cointegration
def correlation_matrix_from_tickers(tickers: list, start: str, end: str = None) -> pd.DataFrame:
    data = yf.download(tickers, start=start, end=end, progress=False, auto_adjust=False)["Close"]
    if isinstance(data, pd.Series): data = data.to_frame()
    rets = data.pct_change().dropna(how="all")
    return rets.corr()

def cointegration_test_pair(ticker_a: str, ticker_b: str, start: str, end: str = None) -> Dict[str, Any]:
    import statsmodels.tsa.stattools as ts
    a = yf.download(ticker_a, start=start, end=end, progress=False)["Close"].dropna()
    b = yf.download(ticker_b, start=start, end=end, progress=False)["Close"].dropna()
    df = pd.concat([a, b], axis=1, join="inner").dropna()
    res = ts.coint(df.iloc[:, 0], df.iloc[:, 1])
    tstat, pvalue = float(res[0]), float(res[1])
    crit = {"1%": res[2][0], "5%": res[2][1], "10%": res[2][2]}
    return {"tstat": tstat, "pvalue": pvalue, "critical_values": crit}

# Mean reversion
def half_life_mean_reversion(series: pd.Series) -> float:
    s = series.dropna()
    if len(s) < 50: return np.nan
    y = s.iloc[1:].values
    x = sm.add_constant(s.shift(1).iloc[1:].values)
    b = sm.OLS(y, x).fit().params[1]
    if b <= 0 or b >= 1: return np.nan
    return float(-np.log(2) / np.log(b))

def hurst_exponent(series: pd.Series, min_window: int = 10, max_window: int = 100) -> float:
    s = series.dropna().values
    if len(s) < max_window*2: return np.nan
    rs = []
    sizes = range(min_window, max_window)
    for w in sizes:
        if len(s) // w < 2: continue
        chunks = s[: (len(s)//w)*w].reshape(-1, w)
        rng = chunks.max(axis=1) - chunks.min(axis=1)
        std = chunks.std(axis=1, ddof=1) + 1e-12
        rs.append(np.log((rng/std).mean()))
    if not rs: return np.nan
    x = sm.add_constant(np.log(np.array(list(sizes)[:len(rs)])))
    h = sm.OLS(np.array(rs), x).fit().params[1]
    return float(h)

# Portfolio helpers
def portfolio_metrics(weights: np.ndarray, returns_df: pd.DataFrame, rf_annual: float = 0.0) -> Dict[str, float]:
    w = np.array(weights).reshape(-1, 1)
    r = returns_df.dropna().astype(float)
    if r.empty:
        return {"ann_return": np.nan, "ann_vol": np.nan, "sharpe": np.nan, "max_drawdown": np.nan, "calmar": np.nan}
    port_ret = (r @ w).squeeze()
    ann = annualized_return_vol(port_ret)
    sr = sharpe_sortino(port_ret, rf_annual=rf_annual)["sharpe"]
    dd = drawdown_series(port_ret).min()
    calmar = ann["ann_return"] / abs(dd) if (pd.notna(ann["ann_return"]) and pd.notna(dd) and dd < 0) else np.nan
    return {"ann_return": ann["ann_return"], "ann_vol": ann["ann_vol"], "sharpe": float(sr), "max_drawdown": float(dd), "calmar": float(calmar)}

def equal_weight(n: int) -> np.ndarray:
    return np.ones(n) / n

def risk_parity_weights(returns_df: pd.DataFrame, window: int = 60) -> np.ndarray:
    r = returns_df.dropna().astype(float)
    if r.shape[0] < window: window = r.shape[0]
    vol = r.tail(window).std(ddof=1)
    inv = 1.0 / (vol.replace(0, np.nan))
    w = inv / inv.sum()
    return w.fillna(0).values

def brute_force_efficient_frontier(returns_df: pd.DataFrame, n_points: int = 50, rf_annual: float = 0.0, seed: int = 42) -> pd.DataFrame:
    rng = np.random.default_rng(seed)
    n = returns_df.shape[1]
    rows = []
    for _ in range(n_points):
        w = rng.random(n); w = w / w.sum()
        m = portfolio_metrics(w, returns_df, rf_annual=rf_annual)
        rows.append({"ann_return": m["ann_return"], "ann_vol": m["ann_vol"], "sharpe": m["sharpe"], "weights": w})
    return pd.DataFrame(rows)


# ============================================================
# 3) NON-INVASIVE WRAPPERS you can call from your pipeline
# ============================================================

def _load_prices(ticker: str, start: str, end: Optional[str] = None) -> pd.DataFrame:
    df = yf.download(ticker, start=start, end=end, progress=False, auto_adjust=False)
    if df is None or df.empty:
        raise ValueError(f"No price data for {ticker}")
    df.index = pd.to_datetime(df.index)
    return df

def technical_quant_summary(ticker: str, benchmark: str, start: str, end: Optional[str] = None, rf_annual: float = 0.0) -> Dict[str, Any]:
    """
    Price-based analytics for a single ticker vs benchmark — plug this into your existing quant function.
    Returns a dict with returns, risk, tails, CAPM, rolling beta, some signals.
    """
    px = _load_prices(ticker, start, end)
    bx = _load_prices(benchmark, start, end)

    r  = simple_returns(px["Close"])
    rb = simple_returns(bx["Close"])

    capm = capm_beta_alpha(r, rb)
    beta_roll = rolling_beta(r, rb, window=60)

    out = {
        "latest_close": float(px["Close"].iloc[-1]),
        "ann": annualized_return_vol(r),
        "sharpe_sortino": sharpe_sortino(r, rf_annual=rf_annual),
        "drawdown": max_drawdown_info(r),
        "tails": {
            "VaR95": value_at_risk(r, 0.05),
            "CVaR95": conditional_var(r, 0.05),
            "omega@0": omega_ratio(r, 0.0),
            "tail_ratio": tail_ratio(r),
            "calmar": calmar_ratio(r),
        },
        "capm": capm,
        "rolling": {
            "vol20": float(rolling_volatility(r, 20).iloc[-1]) if r.shape[0] >= 20 else np.nan,
            "sharpe60": float(rolling_sharpe(r, 60, rf_annual).iloc[-1]) if r.shape[0] >= 60 else np.nan,
            "beta60_last": float(beta_roll.iloc[-1]) if not beta_roll.empty else np.nan
        },
        "signals": {
            "bbands_z60": zscore_last(px["Close"], 60),
            "momentum_12m_1m": momentum_12m_1m(px["Close"]),
            "atr14": float(average_true_range(px, 14).iloc[-1]) if not px.empty else np.nan,
            "hurst_close": hurst_exponent(px["Close"]),
            "half_life_close": half_life_mean_reversion(px["Close"])
        }
    }
    return out

def full_ticker_analysis(ticker: str, benchmark: str = "SPY", start: str = "2022-01-01", end: Optional[str] = None, rf_annual: float = 0.0, prefer_period: str = "annual") -> Dict[str, Any]:
    """
    Convenience helper that bundles fundamentals + technical quant.
    Safe to ignore if your pipeline already orchestrates these separately.
    """
    fundamentals = Fundamental_Analysis_Statements(ticker, prefer_period=prefer_period)
    quant = technical_quant_summary(ticker, benchmark, start, end, rf_annual=rf_annual)
    return {
        "ticker": ticker,
        "fundamental": fundamentals,
        "quant": quant
    }

In [76]:
def Fundamental_Analysis(ticker):
  """
  JSON serializable strings for Analysis statements so that it is simple for the LLMs to consume.
  Incorporates both fundamental and quantitative analysis.
  """
  ticker = ticker.strip().upper()
  try:
    # Perform fundamental analysis
    fundamental_result = Fundamental_Analysis_Statements(ticker)

    # Perform comprehensive quantitative analysis using the new function
    quant_result = technical_quant_summary(ticker, benchmark="SPY", start="2022-01-01") # Using SPY as a default benchmark and a fixed start date

    # Combine the results
    output = {
        'ticker': ticker,
        'fundamental_summary':{
            'marketCap': fundamental_result['ratios'].get('marketCap'),
            'trailingPE': fundamental_result['ratios'].get('trailingPE'),
            'currentPrice': fundamental_result['ratios'].get('currentPrice'),
            'trailingEps': fundamental_result['ratios'].get('trailingEps'),
            'PE Ratio': fundamental_result['ratios'].get('pe_ratio'),
            'ROE Ratio (%)': fundamental_result['ratios'].get('ROE'),
            'Operating Margin (%)': fundamental_result['ratios'].get('operating_margin'),
            'Net Profit Margin (%)': fundamental_result['ratios'].get('net_profit_margin'),
            'Current Ratio': fundamental_result['ratios'].get('current_ratio'),
            'Debt-to-Equity Ratio': fundamental_result['ratios'].get('debt_to_equity_ratio'),
            'ROA (%)': fundamental_result['ratios'].get('ROA'), # Added ROA
            'EBITDA Margin (%)': fundamental_result['ratios'].get('ebitda_margin'), # Added EBITDA Margin
            'FCF Margin (%)': fundamental_result['ratios'].get('fcf_margin'), # Added FCF Margin
            'Interest Coverage': fundamental_result['ratios'].get('interest_coverage'), # Added Interest Coverage
            'Debt-to-EBITDA': fundamental_result['ratios'].get('debt_to_ebitda'), # Added Debt/EBITDA
            'EV-to-EBITDA': fundamental_result['ratios'].get('ev_to_ebitda'), # Added EV/EBITDA
            'ROIC (%)': fundamental_result['ratios'].get('ROIC') # Added ROIC
        },
        'quantitative_summary': quant_result,
        'notes': fundamental_result.get('notes', [])
    }
    # Use json.dumps for clean structured output for the LLM
    return json.dumps(output, indent=2)
  except Exception as e:
    return json.dumps({'error': str(e)})

#### 3. Configuring and Deploying Agents :

In [77]:
model = ChatGroq(
    model = "llama-3.1-8b-instant",
    api_key = os.environ["GROQ_API_KEY"],
    max_tokens = 512, # Maximum number of that can be generated by the model over a single run
    temperature = 0.1, # To ensure precise response. It indicates randomness of token generation where a lower value results in more deterministic token generation.
    tool_choice = "none",
    tools = []
)

/usr/local/lib/python3.12/dist-packages/pydantic/main.py:253: UserWarning: WARNING! tool_choice is not default parameter.
                    tool_choice was transferred to model_kwargs.
                    Please confirm that tool_choice is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)
/usr/local/lib/python3.12/dist-packages/pydantic/main.py:253: UserWarning: WARNING! tools is not default parameter.
                    tools was transferred to model_kwargs.
                    Please confirm that tools is what you intended.
  validated_self = self.__pydantic_validator__.validate_python(data, self_instance=self)


Configuring Agents :

##### Supervisor Implementation (with Reasoning Chains) and Evaluator-Optimizer

In [78]:
# News Data Agent (UPDATED PROMPT)
news_analysis_expert = create_react_agent(
    model = model,
    tools = [Financial_News],
    name = "news_analysis_expert",
    prompt = ("You analyze financial news for given stock symbols. Your task is to extract key topics and summarize the **sentiment** (positive, negative, neutral) and **implication** (e.g., product launch, regulatory change, earnings) of each news article. "
              "Return structured insights ONLY, preferably a JSON list of summaries and their sentiment/implication. This fulfills the Prompt Chaining requirement for extraction and summarization."
    )
)

# Finance Data Analysis Agent (No Change)
finance_analysis_expert = create_react_agent(
    model = model,
    tools = [Financial_Statements],
    name = "finance_analysis_expert",
    prompt = "You are an expert in stock market data."
             "You are a financial statements expert. Always return structured data from Financial_Statements for the requested ticker. Avoid opinions.Do not speculate beyond the data. "
)

# Quantitative Analysis Agent (Updated)
quant_expert = create_react_agent(
    model = model,
    tools = [Fundamental_Analysis, Fundamental_Analysis_Statements, full_ticker_analysis],  # All tools explicitly listed
    name = "quant_expert",
    prompt = (
        "You are a quantitative analyst specializing in both fundamental and technical analysis. "
        "You have access to the following tools: Fundamental_Analysis, Fundamental_Analysis_Statements, and full_ticker_analysis. "
        "Use ALL available tools when relevant to extract structured financial data. "
        "Your responsibilities include computing and reporting key metrics such as P/E ratio, ROE, operating margins, "
        "Current Ratio, Debt-to-Equity Ratio, Net Profit Margin, annualized return, volatility, Sharpe ratio, and drawdown. "
        "Generate visuals whenever supported by the tools. "
        "Always return actual structured values directly from tool outputs—do NOT speculate or provide subjective recommendations. "
        "When possible, prioritize full_ticker_analysis to provide a comprehensive combined overview of both quantitative and fundamental metrics."
    )
)

##### Sub-Agent wrapping as tools for Supervisor :

In [79]:
def run_news_analysis(query: str) -> str:
    """Run the News Analysis Agent"""
    result = news_analysis_expert.invoke({"input": query})
    return result["output"]

def run_finance_analysis(query: str) -> str:
    """Run the Financial Statements Agent"""
    result = finance_analysis_expert.invoke({"input": query})
    return result["output"]

def run_quant_analysis(query: str) -> str:
    """Run the Fundamental/Quantitative Agent"""
    result = quant_expert.invoke({"input": query})
    return result["output"]

# Wrap as structured tools
news_tool = StructuredTool.from_function(run_news_analysis)
finance_tool = StructuredTool.from_function(run_finance_analysis)
quantitative_tool = StructuredTool.from_function(run_quant_analysis)

In [80]:
#### SUPERVISOR AGENT ####
market_research_supervisor = create_react_agent(
    model = model,
    tools = [news_tool, finance_tool, quantitative_tool], # Can fetch data when required
    prompt = (
        "You are a financial market supervisor managing three expert agents: news_analysis_expert, finance_analysis_expert, and quant_expert."
        "Your job is to analyze user query and decide which agent(s) to call or invoke in order."
        "Use the following reasoning chain to gather relevant data:"
        "1. Determine the type of information needed (news, financial statements, or quantitative analysis including fundamental ratios and technical indicators).\n"
        "2. Call only the required agents for the given user query.\n"
        "3. Collect structured outputs from the agents.\n"
        "4. Analyze the collected data, including both fundamental and quantitative metrics, to produce a final structured report."
        "Utilize the data analysis done on behalf of the above-mentioned agents to create a detailed investment thesis to address the user's request."
        "Always reference actual data from the agent outputs, citing specific metrics and values from both the fundamental and quantitative summaries."
        "Please back your assertions with substantial data and analysis. Do NOT generate unsupported claims or general statements like 'strong balance sheet'."
        "Provide factual analysis grounded in the agents' data."
        "You refrain from providing direct 'Buy' or 'Sell' recommendations to comply with legal regulations."
        "Example reasoning: 'User asked about NVDA financial ratios and quantitative indicators -> call quant_expert and finance_analysis_expert only.'"

    )
)

In [81]:
# Creating Evaluator Agent (UPDATED PROMPT)
evaluator_agent = create_react_agent(
    model = model,
    tools = [], # works only with supervisor output
    name = "evaluator_agent",
    prompt=(
        "You are an **Auditor and Quality Control Evaluator** for financial reports. Your role is to critically assess the financial report generated by the Supervisor Agent. "
        "Your output must be a concise, structured critique. If the report is complete and well-supported, state 'REPORT IS COMPLETE AND WELL-SUPPORTED'. Otherwise, provide feedback on *all* points below: \n"
        "- **Completeness:** Are all key metrics (especially the full set of financial ratios like P/E, ROE, Current Ratio, D/E Ratio, and Margins) explicitly included and cited with numerical data? Are news insights included if relevant?\n"
        "- **Consistency & Support:** Are the financial assertions and analysis statements directly supported by the *actual figures* cited from the agent outputs? Are there any unsupported claims? \n"
        "- **Clarity and Structure:** Is the report well-organized, easy to read, and does it directly answer the user's initial query? Is the tone professional?\n"
        "Generate structured feedback emphasizing required improvements to make the final report fully compliant and insightful."
    )
)

In [82]:
# Creating Optimizer Agent
optimizer_agent = create_react_agent(
    model = model,
    tools = [news_tool, finance_tool, quantitative_tool], # Can fetch data when required
    name = "optimizer_agent",
    prompt = (
        "You are an optimizer of financial reports. \n"
        "Using evaluator feedback, refine the report by: \n"
        "- Adding missing metrics\n"
        "- Improving clarity and organization\n"
        "- Incorporating additional agent outputs if required\n"
        "Ensure final report is fully grounded on actual agent data."
    )
)

#### 4. Running the Application:

In [83]:
# Supervisor Agent implementation with Reasoning Chain and Evaluator-Optimizer
def autonomous_stock_analysis(ticker, max_iterations=3):
  stock_query = {
      "messages":[{"role":"user","content":f"Analyze {ticker}'s financials and provide an investment summary."}]
  }
  supervisor_response = market_research_supervisor.invoke(stock_query)['messages'][-1].content
  final_report = supervisor_response
  iteration = 0

  while iteration < max_iterations:
    iteration = iteration + 1

    evaluator_output = evaluator_agent.invoke({
            "messages": [
                {"role": "user", "content": final_report}
            ]
        })['messages'][-1].content

    if "No issues" in evaluator_output or "complete" in evaluator_output.lower():
            break
    final_report = optimizer_agent.invoke({
            "messages": [
                {"role": "user", "content": f"Refine the following report based on feedback:\nReport:\n{final_report}\nEvaluator Feedback:\n{evaluator_output}"}
            ]
        })['messages'][-1].content

  return final_report

In [84]:
appl_report = autonomous_stock_analysis('APPL')
print(appl_report)

To analyze APPL's (Apple Inc.) financials and provide an investment summary, I will follow the reasoning chain:

1. Determine the type of information needed: Since the user asked for an analysis of APPL's financials, I will need to gather financial statement data, including income statements, balance sheets, and cash flow statements.

2. Call the required agents: I will call the finance_analysis_expert to gather financial statement data and the quant_expert to analyze quantitative metrics.

3. Collect structured outputs from the agents:

Finance_analysis_expert output:
- Income Statement:
  - Revenue: $365.96 billion (2022)
  - Net Income: $94.68 billion (2022)
  - Gross Margin: 38.2% (2022)
- Balance Sheet:
  - Total Assets: $384.5 billion (2022)
  - Total Liabilities: $245.5 billion (2022)
  - Equity: $139 billion (2022)
- Cash Flow Statement:
  - Operating Cash Flow: $94.7 billion (2022)
  - Capital Expenditures: $14.3 billion (2022)

Quant_expert output:
- Fundamental Ratios:
  - P